In [ ]:
!pip install pymystem3

In [ ]:
!pip install natasha

In [ ]:
!pip install spacy

In [ ]:
! python -m spacy download ru_core_news_sm

## Домашнее задание номер 2

На последнем семинаре мы проанализировали несколько различных морфологических теггеров. Как же узнать, какой использовать? Давайте сравним их качество!

В этой домашке вам будет нужно найти тексты на русском языке (размер корпуса не менее 200 слов), 
в которых  будут какие-то трудные или неоднозначные для POS теггинга моменты и разметить их вручную 
– с помощью этих текстов мы будем оценивать качество работы наших теггеров. В текстах размечаем только части речи, ничего больше!
1. (1 балл) Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки (какие моменты вы тут считаете трудными для автоматического посттеггинга и почему, в этом вам может помочь второй ридинг). Не забывайте, что разные теггеры могут использовать разные тегсеты: напишите комментарий о том, какой тегсет вы берёте для разметки и почему.

Текст в файле **sents.txt**, разметка текста в файле **gold_standard.txt**. 

Трудные для постеггинга моменты в моём тексте:
- аббревиатуры *ООП*, *ВПО*, *ФГОАУ* и т.п. -- их можно не распознать совсем или спутать с другими частями речи, например, при соответствующих окончаниях
- употребительные сокращения *ст. научн. сотр.* и т.п. -- этих сокращений вполне вероятно нет в словарях моделей и, особенно когда за ними скрываются не существительные, это будет трудный случай для моделей
- составной предлог *за счет*, который я решила делить на две части в угоду орфографии, но помечать обе части как предлог.
- конверсия *родные*, *исполняющий* в И. о., *обратное* в *в обратном*, *данные*
- слова через дефис типа *день-другой*, *ну-ка*, *ходить-бродить* - возникает в первую очередь вопрос, как их делить на токены, а затем, соответственной, какую часть речи приписывать (результат будет различным в зависимости от результатов токенизации)
- предложение с переставленными буквами внутри слова -- человек может понимать такие предложения, поэтому хотелось бы, чтобы и модели могли
- омоформы вроде *стекла*, *мечи*, *косой*, *коса*, *кося* и пр., которые, очевидно, можно перепутать
- неочевидные демонстративы *прочий*, *некоторый*
- слова, которых скорее всего нет в словарях у использующихся моделей *моджибаке*, *памимимный*, *ведомостичка* и т.п.
- нелитературные формы слов *побежду*, *убедю* -- это формы глаголов, которых наверняка нет в словарях моделей, а мы кроме того знаем из ридинга, что глаголы модели предсказывают с меньшей вероятностью, чем существительные и т.п.

Для разметки я взяла тегсет, основанный на стандарте UD, это удобно, так как этот стандарт на данный момент считается наиболее универсальными. Тем не менее, я внесла некоторые изменения и упрощения, в целом из неочевидного:
- соответствующие имена собственные считаются существительными (позволит верно соотнести категории из UD и из mystem, который, соответственно, считает имена собственные просто существительными)
- вспомогательные глаголы типа *быть* считаются глаголами (это более характерно для русской традиции и позволит избежать сложностей при работе с тегами, более ориентированными на русский язык как у mystem)
- все виды причастий и деепричастий -- также формы глаголов (это удобно, потому что сокращает количество частеречных тегов, кроме того причастия и деепричастия однозначно образовываются от глаголов и невыделение их в отдельные классы более универсально с типологической точки зрения)
- *прочий*, *некоторый*, *один* (в соотв. контексте) -- детерминативы, так как это логично для стандарта UD и в стандарте mystem тоже находит соответствие -- категорию APRO, объединяющую местоимённые прилагательные. Тогда в категории местоимений остаются только местоимённые существительные.

In [1]:
with open("sents.txt", 'r', encoding='utf-8') as file:
    sents = file.read()

sents = sents.split('\n')
sents[:5]

['Финансовое обеспечение деятельности Федерального государственного автономного образовательного учреждения высшего профессионального образования ФГОАУ ВПО Национального исследовательского университета «Высшая школа экономики» (далее НИУ ВШЭ) за счет средств федерального бюджета осуществляет Управление делами Президента Российской Федерации.',
 'К 2018 году 100% ООП ВПО прошли экспертизу с участием международных экспертов.',
 'Под Марксом, в кресло вкресленный с высоким окладом, высок и гладок, сидит облачённый ответственный.',
 'Каждый на месте: невеста — в тресте, кум — в Гум, брат — в наркомат.',
 'Все шире периферия родных, и в ведомостичках узких не вместишь всех сортов наградных — спецставки, тантьемы, нагрузки!']

Создаём список списков токенов и POS-тегов на основе образцовой разметки.

In [2]:
with open("gold_standard.txt", 'r', encoding='utf-8') as file:
    tokens = file.read()
    
tokens = tokens.split('\n')
gold_standard_list = []
for token in tokens:
    gold_standard_list.append(token.split('\t'))

In [3]:
gold_standard_list[:10]

[['Финансовое', 'ADJ'],
 ['обеспечение', 'NOUN'],
 ['деятельности', 'NOUN'],
 ['Федерального', 'ADJ'],
 ['государственного', 'ADJ'],
 ['автономного', 'ADJ'],
 ['образовательного', 'ADJ'],
 ['учреждения', 'NOUN'],
 ['высшего', 'ADJ'],
 ['профессионального', 'ADJ']]

2. (3 балла) Потом вам будет нужно взять три  POS теггера для русского языка (udpipe, stanza, natasha, pymorphy, mystem, spacy, deeppavlov) и «прогнать» текст через каждый из них.

### Mystem

In [4]:
from pymystem3 import Mystem

In [5]:
m = Mystem()
mystem_analysis_list = []
for sent in sents:
    sent_analysis = m.analyze(sent)
    for i in range(len(sent_analysis)):
        # Исключаем из анализа пробелы и пунктуацию
        if 'analysis' in sent_analysis[i]:
            lexeme = sent_analysis[i]['text']
            # Вписываем отдельно слова, для которых анализатор не смог определить часть речи
            if sent_analysis[i]['analysis'] != []:
                morph = sent_analysis[i]['analysis'][0]['gr']
                pos = morph.split('=')[0].split(',')[0]
            else:
                pos = None
            mystem_analysis_list.append([lexeme, pos])

In [6]:
mystem_analysis_list[:5]

[['Финансовое', 'A'],
 ['обеспечение', 'S'],
 ['деятельности', 'S'],
 ['Федерального', 'A'],
 ['государственного', 'A']]

### Natasha

In [7]:
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, Doc

In [8]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [9]:
natasha_analysis = Doc(" ".join(sents))
natasha_analysis.segment(segmenter)
natasha_analysis.tag_morph(morph_tagger)

In [10]:
natasha_analysis.sents[0].morph.tokens[0]

MorphToken(
    text='Финансовое',
    pos='ADJ',
    feats={'Case': 'Nom',
     'Degree': 'Pos',
     'Gender': 'Neut',
     'Number': 'Sing'}
)

### Spacy

In [11]:
import spacy

In [12]:
nlp = spacy.load("ru_core_news_sm")

In [13]:
spacy_doc = nlp(" ".join(sents))
spacy_analysis_list = []
for token in spacy_doc:
    spacy_analysis_list.append([token.text, token.pos_])

In [14]:
spacy_analysis_list[:10]

[['Финансовое', 'ADJ'],
 ['обеспечение', 'NOUN'],
 ['деятельности', 'NOUN'],
 ['Федерального', 'ADJ'],
 ['государственного', 'ADJ'],
 ['автономного', 'ADJ'],
 ['образовательного', 'ADJ'],
 ['учреждения', 'NOUN'],
 ['высшего', 'ADJ'],
 ['профессионального', 'ADJ']]

3. (2 балла) Затем оценим accuracy для каждого теггера. Заметьте, что в разных системах имена тегов и части речи  могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции-конвертера и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции.

In [15]:
def convert_tags(word_pos_tag, model):
    if model.lower() == 'mystem':
        dict_corr = {'A': 'ADJ', 'PR': 'ADP', 'S': 'NOUN', 'V': 'VERB', 'ANUM': 'NUM',
                                      'APRO': 'DET', 'ADVPRO': 'ADV', 'SPRO': 'PRO'}
    elif model.lower() == 'natasha' or model.lower() == 'spacy':
        dict_corr = {'PROPN': 'NOUN', 'AUX': 'VERB', 'CCONJ': 'CONJ', 'SCONJ': 'CONJ', 'PUNCT': None}
    
    if word_pos_tag in dict_corr:
        return(dict_corr[word_pos_tag])
    else:
        return(word_pos_tag)

In [16]:
mystem_conv_analysis_list = []
for mystem_word_analysis in mystem_analysis_list:
    mystem_conv_tag = convert_tags(mystem_word_analysis[1], 'mystem')
    mystem_conv_analysis_list.append([mystem_word_analysis[0], mystem_conv_tag])

In [17]:
mystem_conv_analysis_list[:5]

[['Финансовое', 'ADJ'],
 ['обеспечение', 'NOUN'],
 ['деятельности', 'NOUN'],
 ['Федерального', 'ADJ'],
 ['государственного', 'ADJ']]

In [18]:
natasha_conv_analysis_list = []
for natasha_sent_analysis in natasha_analysis.sents:
    for natasha_word_analysis in natasha_sent_analysis.morph.tokens:
        natasha_conv_word_analysis = [natasha_word_analysis.text,
                                           convert_tags(natasha_word_analysis.pos, 'natasha')]
        if natasha_conv_word_analysis[1] != None:
            natasha_conv_analysis_list.append(natasha_conv_word_analysis)

In [19]:
natasha_conv_analysis_list[:5]

[['Финансовое', 'ADJ'],
 ['обеспечение', 'NOUN'],
 ['деятельности', 'NOUN'],
 ['Федерального', 'ADJ'],
 ['государственного', 'ADJ']]

In [20]:
spacy_conv_analysis_list = []
for spacy_word_analysis in spacy_analysis_list:
    spacy_conv_tag = convert_tags(spacy_word_analysis[1], 'spacy')
    if spacy_conv_tag != None:
        spacy_conv_analysis_list.append([spacy_word_analysis[0], spacy_conv_tag])

In [21]:
spacy_conv_analysis_list[:10]

[['Финансовое', 'ADJ'],
 ['обеспечение', 'NOUN'],
 ['деятельности', 'NOUN'],
 ['Федерального', 'ADJ'],
 ['государственного', 'ADJ'],
 ['автономного', 'ADJ'],
 ['образовательного', 'ADJ'],
 ['учреждения', 'NOUN'],
 ['высшего', 'ADJ'],
 ['профессионального', 'ADJ']]

При сопоставлении самое сложное -- разобраться со съехавшими токенами.

In [22]:
len(gold_standard_list), len(mystem_conv_analysis_list), len(natasha_conv_analysis_list), len(spacy_conv_analysis_list)

(231, 231, 232, 240)

In [26]:
import csv   
    
with open('analysis_comparison.tsv', 'w', newline='', encoding='utf-8') as file:
    longest_analysis_len = max(len(gold_standard_list), len(mystem_conv_analysis_list),
                               len(natasha_conv_analysis_list), len(spacy_conv_analysis_list))
    for i in range(longest_analysis_len):
        if len(gold_standard_list) <= i:
            gold_standard_analysis = ['', '']
        else:
            gold_standard_analysis = gold_standard_list[i]
        if len(mystem_conv_analysis_list) <= i:
            mystem_analysis = ['', '']
        else:
            mystem_analysis = mystem_conv_analysis_list[i]
        if len(natasha_conv_analysis_list) <= i:
            natasha_analysis = ['', '']
        else:
            natasha_analysis = natasha_conv_analysis_list[i]
        spacy_analysis = spacy_conv_analysis_list[i]
        string = [gold_standard_analysis[0], gold_standard_analysis[1],
                  mystem_analysis[0], mystem_analysis[1],
                  natasha_analysis[0], natasha_analysis[1],
                  spacy_analysis[0], spacy_analysis[1]]
        tsv_output = csv.writer(file, delimiter='\t')
        tsv_output.writerow(string)

Далее наполовину вручную разбираемся с произошедшими из-за различных алгоритмов токенизации у разных моделей сдвигами в соответствии токенов. Если морфологизатор неверно делил слово на токены, то брался pos-тег последнего токена.

In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('analysis_comparison.tsv', sep='\t', encoding='utf-8', header=None)
df.columns = ['gold_standard_token', 'gold_standard_tag',
              'mystem_token', 'mystem_tag',
              'natasha_token', 'natasha_tag',
              'spacy_token', 'spacy_tag']
df.loc[33:,['mystem_token', 'mystem_tag']] = df.loc[33:,['mystem_token', 'mystem_tag']].shift(1)
df.loc[35:,['mystem_token', 'mystem_tag']] = df.loc[35:,['mystem_token', 'mystem_tag']].shift(1)
df.loc[36:,['natasha_token', 'natasha_tag', 'spacy_token', 'spacy_tag']] = \
                df.loc[36:,['natasha_token', 'natasha_tag', 'spacy_token', 'spacy_tag']].shift(-1)
df.loc[89:,['mystem_token', 'mystem_tag']] = df.loc[89:,['mystem_token', 'mystem_tag']].shift(-1)
df.loc[89:,['spacy_token', 'spacy_tag']] = df.loc[89:,['spacy_token', 'spacy_tag']].shift(-2)
df.loc[118:,['spacy_token', 'spacy_tag']] = df.loc[118:,['spacy_token', 'spacy_tag']].shift(-1)
df.loc[149:,['spacy_token', 'spacy_tag']] = df.loc[149:,['spacy_token', 'spacy_tag']].shift(-1)
df.loc[150:,['spacy_token', 'spacy_tag']] = df.loc[150:,['spacy_token', 'spacy_tag']].shift(-2)
df.loc[165:,['mystem_token', 'mystem_tag']] = df.loc[165:,['mystem_token', 'mystem_tag']].shift(-1)
df.loc[165:,['spacy_token', 'spacy_tag']] = df.loc[165:,['spacy_token', 'spacy_tag']].shift(-2)
df = df.loc[:230, ['gold_standard_token', 'gold_standard_tag', 'mystem_tag', 'natasha_tag', 'spacy_tag']]
df.tail()

,gold_standard_token,gold_standard_tag,mystem_tag,natasha_tag,spacy_tag
226,косого,ADJ,ADJ,VERB,ADJ
227,косо,ADV,ADV,ADV,ADV
228,косил,VERB,VERB,VERB,VERB
229,косую,ADJ,ADJ,ADJ,NOUN
230,косу,NOUN,NOUN,NOUN,NOUN


In [29]:
from sklearn.metrics import accuracy_score

In [30]:
print('Mystem', accuracy_score(df['gold_standard_tag'], df['mystem_tag'].astype(str)))
print('Natasha', accuracy_score(df['gold_standard_tag'], df['natasha_tag']))
print('Spacy', accuracy_score(df['gold_standard_tag'], df['spacy_tag']))

Mystem 0.8051948051948052
Natasha 0.8398268398268398
Spacy 0.8311688311688312


Итак, из тех постеггеров, которые я сравнивала, лучшим оказался Natasha.

4. (4 балла) Дальше вам нужно взять лучший теггер для русского языка и с его помощью написать функцию (chunker),  которая выделяет из размеченного текста 3 типа n-грамм, соответствующих какому-то шаблону (к примеру не + какая-то часть речи или NP или сущ.+ наречие и тд) В предыдущем дз многие из вас справедливо заметили, что если бы мы могли класть в словарь не только отдельные слова, но и словосочетания, то программа работала бы лучше. Предложите 3 шаблона (слово + POS-тег / POS-тег + POS-тег) запись которых в словарь, по вашему мнению, улучшила бы качество работы программы из предыдущей домашки. Балл за объяснение того, почему именно эти группы вы взяли, балл за создание такого рода чанкера, балл за  за встраивание функции в программу из предыдущей домашки, балл за сравнение качества предсказания тональности с улучшением и без.

Поизучав тексты отзывов к фильмам, которые я использовала, я решила выбрать следующие шаблоны:
- *не* + VERB
- *слишком* + ADV/ADJ
- *хороший* + NOUN

Сочетания глагола с *не* часто используются, чтобы описать, что не понравилось в фильме, ср. *не тянут, не понравился, не рекомендую* и т.п. Сочетания со *слишком* тоже несут отрицательные оттенки значения из-за семантики этого наречия, довольно часто пишут: *слишком много, уныло, плоский* и т.п. *Хороший* + существительное же, наоборот, направлено на выявление положительных отзывов. Довольно часто пишут *хороший фильм, хорошая документалка* и т.п., когда говорят о положительных эмоциях от фильма, при этом просто *хороший* без дополнительных условий на существительное после него допускает чуть больше вариативности в том, что это за словосочетания, ср. *ни к чему хорошему не приведёт* в отрицательном отзыве.

In [31]:
from natasha import MorphVocab

In [32]:
def chunker(text):
    
    segmenter = Segmenter()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    morph_vocab = MorphVocab()

    tagged_text = Doc(text)
    tagged_text.segment(segmenter)
    tagged_text.tag_morph(morph_tagger)

    tagged_list = []
    for tagged_word in tagged_text.tokens:
        tagged_word.lemmatize(morph_vocab)
        tagged_conv_words = [tagged_word.lemma, convert_tags(tagged_word.pos, 'natasha')]
        if tagged_conv_words[1] != None:
            tagged_list.append(tagged_conv_words)
    
    df = pd.DataFrame(tagged_list)
    df.columns = ['lemma', 'tag']
    df['next_lemma'] = df['lemma'].shift(-1)
    df['next_tag'] = df['tag'].shift(-1)
    
    return df[(df['lemma'] == 'не') & (df['next_tag'] == 'VERB')], \
        df[(df['lemma'] == 'слишком') & ((df['next_tag'] == 'ADV') | (df['next_tag'] == 'ADJ'))], \
        df[(df['lemma'] == 'хороший') & (df['next_tag'] == 'NOUN')]

In [33]:
chunker(" ".join(sents))[0]

,lemma,tag,next_lemma,next_tag
80,не,PART,вместить,VERB
94,не,PART,дождаться,VERB
127,не,PART,иеемт,VERB
161,не,PART,хотеть,VERB
173,не,PART,бывать,VERB
180,не,PART,бывать,VERB


Встроить функцию в программу из предыдущей домашки я решила в ноутбуке предыдущей домашки в новом последнем разделе -- файл Hometask_1_better. Там же можно увидеть сравнение качества предсказания тональности с улучшением и без. Конечно, многое зависит от случайности, так как при таком количестве тестирующих примеров accuracy не является сильно точным значением, однако на последнем моём тесте это улучшение действительно улучшило качество: с 0.65 до 0.7.